### Stage 3: Analysis of the key reporting metrics and final feature selection.
---
### Tree features evaluation results
---
### ✅ High-Quality Features (Low Missing + Low Zero Rate)

| Feature                    | Missing Rate  | Zero Rate | Description                                   |
|----------------------------|---------------|-----------|-----------------------------------------------|
| int_rate                   | 0.0%          | 0.0%      | Interest rate (decimal)                       |
| fico_average               | 0.0%          | 0.0%      | Average FICO score                            |
| grade_encoded              | 0.0%          | 0.0%      | LendingClub ordinal credit grade              |
| sub_grade_encoded          | 0.0%          | 0.0%      | Fine-grained credit subgrade                  |
| term_tree                  | 0.0%          | 0.0%      | Categorical loan term                         |
| loan_to_installment_ratio  | 0.0%          | 0.0%      | Loan amount vs. installment ratio             |
| fico_risk_band             | 0.0%          | —         | Binned FICO risk categories                   |
| income_band_tree           | 0.0%          | 10.0%     | Income decile bins                            |
| is_high_util_tree          | 0.0%          | 50.1%     | Flag for high credit utilization              |
| is_high_income_tree        | 0.0%          | 50.9%     | Flag for high income                          |
| is_high_dti_tree           | 0.0%          | 50.0%     | Flag for high debt-to-income ratio            |
| revol_util_tree            | 0.0%          | 0.6%      | Cleaned revolving utilization                 |
| purpose_risk_score         | 0.0%          | 0.0%      | Purpose-mapped risk score                     |


### ⚠️ Features with High Missing Rates

| Feature                         | Missing Rate | Description                                 |
|---------------------------------|--------------|---------------------------------------------|
| annual_inc_joint                | 93.1%        | Joint applicant income                      |
| dti_joint                       | 93.1%        | Joint applicant DTI                         |
| avg_cur_bal_missing             | 97.6%        | Missing flag for average current balance    |
| tot_cur_bal_missing             | 97.6%        | Missing flag for total current balance      |
| num_tl_op_past_12m_missing      | 97.6%        | Trade lines opened in past 12m (missing)    |
| mths_since_recent_inq_missing   | 87.3%        | Missing flag for recent inquiries           |
| hardship_dpd_filled             | 99.1%        | Days past due during hardship               |
| was_late_before_hardship        | 99.8%        | Loan already late before hardship           |
| loan_status_binary              | ~100%        | ⚠️ Critical target field — likely dropped!  |


### 🚫 Low-Variance / Sparse Features (Zero-Dominant or Unused)

| Feature                   | Zero Rate | Notes                                  |
|---------------------------|-----------|----------------------------------------|
| purpose_nan               | 100%      | Always non-null — redundant            |
| high_payment_burden       | 99.1%     | Almost always 0 — very sparse          |
| was_late_before_hardship  | 99.8%     | Rarely true — possibly drop            |
| recent_major_derog_flag   | 95.1%     | Sparse flag, low utility               |


### 📈 Binary Tree Features with Split Potential

| Feature                   | Zero Rate | Notes                                |
|---------------------------|-----------|--------------------------------------|
| has_derogatory            | 69.7%     | Indicates any derogatory history     |
| has_recent_delinq_tree    | 82.5%     | Recency of delinquencies             |
| has_delinq_tree           | 82.2%     | Any past delinquency                 |
| purpose_high_risk         | 1.7%      | Differentiates low-risk purposes     |
| is_high_util_tree         | 50.1%     | Balanced split for utilization       |
| is_high_income_tree       | 50.9%     | Balanced split for income            |
| is_high_dti_tree          | 50.0%     | Balanced split for DTI               |
